In [124]:
# This program uses scikits gridsearch cv and helps to find the best set of parameters for a classifier. This is 
# generally used for tuning up the classifier paramters.

In [41]:
import pandas as pd
import numpy as np
import string as str
from os import listdir
from os.path import isfile, join
from  datetime import datetime
from os import listdir
from os.path import isfile, join

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk.stem
from sklearn.decomposition import PCA

from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.cross_validation import KFold

import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
%matplotlib inline  
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
import AppVectorizerModule as ATV

from sklearn.decomposition import PCA

In [42]:
# Make a gridsearchresult folder
resultpath='/home/ayush/Desktop/gsresult/'
#resultpath='../gsresult/'

In [43]:
# A function to return parameters based on classifier
def getParamGrid(clfname):
    clfname=getClassifierName(clfname)
    param_grid={}
    if clfname=='RandomForestClassifier':   
        param_grid = { 
            'n_estimators': [50,100,200,500],
            'max_features': [None, 'sqrt', 'log2'],
            'criterion' : ['gini','entropy'],
            'oob_score' : [True,False],
            'random_state' : [0,10,20,30]
        }
    elif clfname=='DecisionTreeClassifier':
        param_grid = { 
            'max_features': ['auto', 'sqrt', 'log2'],
            'criterion' : ['gini','entropy'],
            'random_state' : [0,10,20,30],
            'splitter' : ['random','best']
        }
    elif clfname=='LinearSVC':
        param_grid = { 
            'tol' : [0.0001,0.0002,0.0003,0.004],
            'multi_class' : ['ovr','crammer_singer'],
            'fit_intercept' : [True,False],
            'random_state': [0,10,20,30],
            'max_iter' : [1000,1500],
            'C':[1.0,10.0,100.0]
        }
    elif clfname=='LogisticRegression':
          param_grid = { 
            'random_state': [0,10,20,30],
            'max_iter' : [1000,1500],
            'multi_class' : ['ovr'],
            'tol' : [0.0001,0.0002,0.0003,0.004],
            'solver' : ['newton-cg', 'lbfgs', 'liblinear'],
            'fit_intercept' : [True,False],
            'C':[1.0,10.0,100.0]
        }
    elif clfname=='SGDClassifier':
        param_grid = { 
           'alpha':[0.0001,0.0002,0.0005,0.001,0.002],
            'fit_intercept' : [True,False],
            'n_iter' : [5,10,20,50],
            'shuffle' : [True,False],
            'random_state': [0,10,20,30],
            'warm_start':[True,False]
        }
    elif clfname=='RidgeClassifier':
        param_grid = { 
            'alpha':[0.0001,0.0002,0.0005,0.001,0.002],
            'copy_X':[True,False],
            'fit_intercept' : [True,False],
            'max_iter' : [500,1000],
            'normalize' : [True,False],
            'solver': ['auto', 'svd', 'cholesky','lsqr','sparse_cg'],
            'tol' : [0.0001,0.0002,0.0003,0.0004],
        }
    elif clfname=='KNeighborsClassifier':
        param_grid = { 
            'n_neighbors':[5,10,15,20],
            'weights':['uniform','distance'],
            'algorithm' : ['auto','ball_tree', 'kd_tree','brute'],
            'metric':['euclidean','manhattan','chebyshev','minkowski']
        }
    return param_grid

def getClassifierName(clf):
    if type(clf).__name__ != 'OneVsRestClassifier':
        #print type(clf).__name__
        return type(clf).__name__
    else:
        #print type(clf.estimator).__name__ 
        return type(clf.estimator).__name__

In [44]:
#A function to printResults
def printResults(resultpath,clfname,grid_scores,best_params,clf_report,best_score):
    print resultpath+clfname
    gridResults=open(resultpath+'_Result_'+clfname+'.txt', 'w')
    print "Grid Scores:\n"
    gridResults.write("Grid Scores:\n")
    for params, mean_score, scores in grid_scores:
            print("%0.3f (+/-%0.03f) for %r"% (mean_score, scores.std() * 2, params))
            gridResults.write("\n%0.3f (+/-%0.03f) for %r"% (mean_score, scores.std() * 2, params))
    print "\nBest Score:\n"
    gridResults.write("\nBest Score:\n")
    print "\nBest Params:\n"
    gridResults.write("\nBest Params:\n")
    print best_score
    gridResults.write("%s"%(best_score))
    print best_params
    gridResults.write("%s"%(best_params))
    print "\nClassification Report:\n"
    gridResults.write("\nClassification Report:\n")
    print(clf_report)
    gridResults.write("%s"%(clf_report))

In [45]:
#single file
path='/home/ayush/Desktop/BigData_ProjectData/PlayStoreCsv/data/big_data1.csv'
df= pd.read_csv(path,sep=',',header=0)

#whole data
'''dfs=[]
path='../data/'
onlyfiles = [ f for f in listdir(path) if isfile(join(path,f)) and f.endswith( 'csv' ) ]
print onlyfiles
for eachFile in onlyfiles:
    path1='../data/'+eachFile
    df1= pd.read_csv(path1,sep=',',header=0)
    print df1.shape[0]
    dfs.append(df1)
df=pd.concat(dfs)
df.shape[0]'''

"dfs=[]\npath='../data/'\nonlyfiles = [ f for f in listdir(path) if isfile(join(path,f)) and f.endswith( 'csv' ) ]\nprint onlyfiles\nfor eachFile in onlyfiles:\n    path1='../data/'+eachFile\n    df1= pd.read_csv(path1,sep=',',header=0)\n    print df1.shape[0]\n    dfs.append(df1)\ndf=pd.concat(dfs)\ndf.shape[0]"

In [47]:
def mapBooleanColumns(x):
    if x==True:
        return 1
    else:
        return 0
    
def mappingScores(x):
    if x>0 and x<=1:
        return 1
    elif x>1 and x<=2:
        return 2
    elif x>2 and x<=3:
        return 3
    elif x>3 and x<=4:
        return 4
    else:
        return 5

def mapDates(d):
    d=d[:10]
    d1= datetime.now()
    d2=datetime.strptime(d, "%Y-%m-%d")
    return (d1.year - d2.year)*12 + d1.month - d2.month    
    
def seperateInstallations(col_name):
    return_data_type_as=None
    ls = col_name.str.split('-').str.get(0).str.strip(' ').str.replace(',','') #series object
    hs = col_name.str.split('-').str.get(1).str.strip(' ').str.replace(',','') #series object
    ls = ls.astype(float).fillna(0.0)
    hs = hs.astype(float).fillna(0.0)
    return ls, hs

def processAppSize(x):
    if x==-1:
        return 0.5
    else:
        return x

In [48]:
# Transforming the data into numerical forms
df.IsTopDeveloper=map(mapBooleanColumns,df.IsTopDeveloper)
df.HaveInAppPurchases=map(mapBooleanColumns,df.HaveInAppPurchases)
df.IsFree=map(mapBooleanColumns,df.IsFree)
df.Score=map(mappingScores,df.Score)
df.PublicationDate=map(mapDates,df.PublicationDate)
df.LastUpdateDate=map(mapDates,df.LastUpdateDate)
df['minInstall'],df['maxInstall']=seperateInstallations(df.Instalations)
df.AppSize=map(processAppSize,df.AppSize)

In [49]:
# Selecting Features
df=df.head(n=3000)
X=df[['Price','PublicationDate','AppSize','LastUpdateDate','IsFree','HaveInAppPurchases','minInstall','maxInstall']]
y=pd.DataFrame(df[['Score']])

In [50]:
# Vectorising the features and merging them to selected numerical features     
min_df=1
max_df=0.9
lowercase=True
stem_vectorizer = ATV.StemmedCountVectorizer(encoding='utf-8',
                                         min_df =min_df,
                                         max_df =max_df,
                                         stop_words='english',
                                         analyzer='word',
                                         lowercase = lowercase)
                
##set filterparameter to your vectorizer
filter_by=["OnlyEng", "AllLang"] #two options are available
count_dialect = True 
n_samples = df.shape[0] #as u choose it.
stem_vectorizer.setfilter_option(filter_by[1],count_dialect)    
# Vectorizing the Category Column
nCategory, fmatrix, column_vectorizer = ATV.vectorize_columnTfIdf(df, 'Category', stem_vectorizer,df.shape[0],True)
# Vectorizing the Name Column
stem_vectorizer.setfilter_option(filter_by[0],count_dialect) 
nName, fmatrix1, column_vectorizer1 = ATV.vectorize_columnTfIdf(df, 'Name', stem_vectorizer,df.shape[0],True)
# Vectorizing the Description Column
nDescription, fmatrix2, column_vectorizer2 = ATV.vectorize_columnTfIdf(df, 'Description',stem_vectorizer,df.shape[0],True)
# Vectorizing the Content Rating Column
nContentRating, fmatrix3, column_vectorizer3 = ATV.vectorize_columnTfIdf(df, 'ContentRating',stem_vectorizer,df.shape[0],True)
# Vectorizing the Developer Column
nDeveloper, fmatrix3, column_vectorizer3 = ATV.vectorize_columnTfIdf(df, 'Developer',stem_vectorizer,df.shape[0],True)

finaldf = pd.concat([X,nCategory,nName,nDescription,nContentRating,nDeveloper],axis=1)

formed dataframe of size:( 3000 , 41 )
formed dataframe of size:( 3000 , 1261 )
formed dataframe of size:( 3000 , 7635 )
formed dataframe of size:( 3000 , 7 )
formed dataframe of size:( 3000 , 602 )


In [38]:
print finaldf.shape,y.shape

(3000, 9554) (3000, 1)


In [51]:
def processClassifier(Xtrain,Xtest,ytrain,ytest,cl,param_grid,resultpath):
    clf = GridSearchCV(estimator=cl, param_grid=param_grid,n_jobs=4,cv=10)
    clf.fit(Xtrain, ytrain)
    ypred=clf.predict(Xtest)

    grid_scores=clf.grid_scores_
    best_params=clf.best_params_
    clf_report=classification_report(ytest,ypred)
    best_score=clf.best_score_
    printResults(resultpath,getClassifierName(cl),grid_scores,best_params,clf_report,best_score)
    

In [63]:
classifiers = [RandomForestClassifier(),LogisticRegression(),SGDClassifier(),RidgeClassifier()
              ,DecisionTreeClassifier(),KNeighborsClassifier()]

xx=finaldf.copy()
pca = PCA(n_components=100)
xx= pca.fit_transform(xx)
yy=y.values
Xtrain, Xtest, ytrain, ytest = train_test_split(xx, yy.ravel(), test_size=0.7, random_state=0)
for clf in classifiers:
    print ("\n>> Start Processing %s"%(getClassifierName(clf)))
    param_grid=getParamGrid(clf)
    processClassifier(Xtrain,Xtest,ytrain,ytest,clf,param_grid,resultpath)
    print ("\n>> End Processing %s"%(getClassifierName(clf)))


>> Start Processing RandomForestClassifier
/home/ayush/Desktop/gsresult/RandomForestClassifier
Grid Scores:

0.679 (+/-0.071) for {'max_features': None, 'n_estimators': 50, 'oob_score': True, 'random_state': 0, 'criterion': 'gini'}
0.671 (+/-0.117) for {'max_features': None, 'n_estimators': 50, 'oob_score': True, 'random_state': 10, 'criterion': 'gini'}
0.668 (+/-0.100) for {'max_features': None, 'n_estimators': 50, 'oob_score': True, 'random_state': 20, 'criterion': 'gini'}
0.663 (+/-0.079) for {'max_features': None, 'n_estimators': 50, 'oob_score': True, 'random_state': 30, 'criterion': 'gini'}
0.679 (+/-0.071) for {'max_features': None, 'n_estimators': 50, 'oob_score': False, 'random_state': 0, 'criterion': 'gini'}
0.671 (+/-0.117) for {'max_features': None, 'n_estimators': 50, 'oob_score': False, 'random_state': 10, 'criterion': 'gini'}
0.668 (+/-0.100) for {'max_features': None, 'n_estimators': 50, 'oob_score': False, 'random_state': 20, 'criterion': 'gini'}
0.663 (+/-0.079) for 

In [58]:
print xx.shape

(3000, 100)


In [62]:
print yy.ravel()

[5 3 5 ..., 5 5 5]
